In [1]:
from typing import Annotated
from dotenv import load_dotenv
load_dotenv()

from typing_extensions import TypedDict
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [2]:
from langchain.chat_models import init_chat_model

In [3]:
llm = init_chat_model("google_genai:gemini-2.0-flash")

class State(TypedDict):
    messages: Annotated[list, add_messages]

def chatbot(state: State) -> State:
    return {"messages": [llm.invoke(state["messages"])]}

builder = StateGraph(State)
builder.add_node("chatbot_node", chatbot)

builder.add_edge(START, "chatbot_node")
builder.add_edge("chatbot_node", END)

graph = builder.compile()

In [5]:
message = {"role": "user", "content": "Who walked on the moon for the first time? Print only the name"}
# message = {"role": "user", "content": "What is the latest price of MSFT stock?"}
response = graph.invoke({"messages":[message]})

response["messages"]

[HumanMessage(content='Who walked on the moon for the first time? Print only the name', additional_kwargs={}, response_metadata={}, id='a7d46571-95d0-43c6-95c8-19dba6028e8d'),
 AIMessage(content='Neil Armstrong', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--9cb17c1a-5232-460e-b0f3-1b28160ec88d-0', usage_metadata={'input_tokens': 14, 'output_tokens': 3, 'total_tokens': 17, 'input_token_details': {'cache_read': 0}})]

In [4]:
state=None
while True:
    in_message=input("You : ")
    if in_message.lower() in ['quit', 'exit']:
        break
    if state is None:
        state : State={
            "messages": [{'role': "user","content":in_message}],
        }
    else:
        state["messages"].append({'role':'user','content':in_message})
    state=graph.invoke(state)
    print("Bot : ", state['messages'][-1].content)

Bot :  LangGraph is a Python library that builds upon LangChain to enable developers to create **stateful, multi-actor conversational applications** with ease. Think of it as a framework for orchestrating complex, graph-structured workflows involving multiple agents or functions, all while maintaining a persistent state that evolves with each interaction.

Here's a breakdown of what makes LangGraph unique and powerful:

**Key Concepts and Features:**

*   **Graph-Based Architecture:** The core idea is to represent your application's logic as a directed graph. Nodes in the graph represent different actors or functions (like LLMs, tools, or data transformations), and edges define the flow of information between them. This graph structure makes it easy to visualize, understand, and modify complex conversational flows.

*   **State Management:** LangGraph manages the "state" of your conversation. This state is a persistent data structure that stores information relevant to the ongoing inte